In [1]:
## single family vs multi-unit
    # most single family homes should be L2
    # some fraction L1? 10%
    # 80% EV owners live in single family homes <- this is harder to code up
    # 2035 60% of EV owners live in single family homes
    # these numbers should come from the stock rollover model - registration data?

    ## multi-unit 
    # some residents wont have access to charging
    # 20% have L2 at home (increase to 75% by 2035) should also charge while out. 
    # 25% work L2 (increase to 50% by 2035) and public but not home
    # remaining % public DCFC/L2 everyone has access to this. 

    ## workplace 
    # what % have access at work?
    # default to 10% have access to work charging, will increase over time
    
    # 50% of civic buildings have charging
    # 10% of industrial 
    # TNCs are either L2 or DCFC

    # use destination_building_use_l2
    
    ## Public 
    # Higher L2 = 19 kw
    # if retail - DCFC
    # if open parking - L2
    # if trip longer than 100 miles, assume next charge is DCFC
    # the lower limit of dwell time should be longer for L2

    # what about light duty fleet vehicles? -> ask Dave


    ### What outputs?
    # number of chargers per type per county. 
    # load curve for each charger type per county. 

In [1]:
import pandas as pd
import numpy as np
import replicaEVSE.load_curve as sim
import os
import joblib
import dask.dataframe as dd

%reload_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', None)

datadir = '../../data/'
# datadir = '../../../data'

mode = 'PRIVATE_AUTO'
simulation_id = 'dev'
test = True

#Created in the EIA_data_download.ipynb notebook
#existing_load=pd.read_csv('../../data/EIA_demand_summary.csv') 
existing_load=pd.read_csv(os.path.join(datadir,'EIA_demand_summary.csv'))
if test:
    df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_subsample.parquet'))
    df = df.loc[df['mode'] == mode]
    df = df.sort_values(by=['person_id', 'start_time']).reset_index(drop=True)
    simulation_id = 'dev'

else: 


    # read in the joined trips and population data sets
    merged_df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_sorted.parquet'))

    # right now, only look at private auto trips
    df = merged_df.loc[merged_df['mode'] == mode]

In [2]:
df

,index,activity_id,person_id,mode,travel_purpose,previous_activity_type,start_time,end_time,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,origin_bgrp_lng,destination_bgrp,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,origin_building_use_l1,origin_building_use_l2,destination_land_use_l1,destination_land_use_l2,destination_building_use_l1,destination_building_use_l2,origin_lat,origin_lng,destination_lat,destination_lng,weekday,household_id,BLOCKGROUP,BLOCKGROUP_work,BLOCKGROUP_school,TRACT,TRACT_work,TRACT_school,age_group,age,sex,race,ethnicity,individual_income_group,individual_income,employment,education,school_grade_attending,industry,household_role,subfamily_number,subfamily_relationship,commute_mode,tenure,migration,household_size,household_income_group,household_income,family_structure,vehicles,building_type,resident_type,language,lat,lng,lat_work,lng_work,lat_school,lng_school,wfh,charge_type
0,4199,7952205927434267893,10000366922410938523,PRIVATE_AUTO,HOME,EAT,0 days 12:41:15,0 days 12:43:31,0.602554,nan,530330068002,47.636260,-122.358964,530330067004,47.635745,-122.347078,commercial,retail,commercial,retail,residential,multi_family,residential,multi_family,47.638249,-122.356634,47.636388,-122.346274,thursday,1732374551127832797,530330067004,nan,nan,53033006700,nan,nan,35_64,62.0,M,white,not_hispanic_or_latino,20000_40000,22176.0,notinlf,some_college,not_attending_school,naics72,head_of_household,0,nan,not_working,renter,same_house,5_person,40000_75000,59308.0,nonfamily_single,1,multiple_units,core,english,47.636388,-122.346274,NaN,NaN,NaN,NaN,under_16_or_notinlf,HOME
1,3575,506829784905170377,10001830118790204821,PRIVATE_AUTO,HOME,EAT,0 days 19:43:00,0 days 19:56:05,1.011157,nan,530610516012,47.844719,-122.299453,530610516021,47.834790,-122.308797,commercial,retail,commercial,retail,mixed_use,single_family,residential,single_family,47.840838,-122.300091,47.834503,-122.313617,thursday,3635700334446387909,530610516021,530350805002,530610516014,53061051602,53035080500,53061051601,18_24,18.0,M,white,not_hispanic_or_latino,0_20000,216.0,employed,high_school,school,naics722513,child,0,nan,carpool,renter,same_house,3_person,40000_75000,41273.0,family_single,2,multiple_units,core,english,47.834503,-122.313617,NaN,NaN,47.845652,-122.315406,worked_in_person,HOME
2,4816,13722390883687412311,10002290586277908152,PRIVATE_AUTO,HOME,EAT,0 days 18:40:12,0 days 18:46:32,2.150550,nan,530330323092,47.675673,-122.126007,530330228023,47.663252,-122.138520,mixed_use,office,commercial,retail,residential,single_family,residential,single_family,47.670172,-122.111026,47.664865,-122.135813,thursday,4129290245843427363,530330228023,nan,nan,53033022802,nan,nan,25_34,34.0,F,asian,not_hispanic_or_latino,lte_0,0.0,notinlf,advanced_degree,not_attending_school,not_working,spouse,0,nan,not_working,owner,same_house,5_person,125000_plus,319710.0,married_couple,3_plus,single_family,core,asian_pacific,47.664865,-122.135813,NaN,NaN,NaN,NaN,under_16_or_notinlf,HOME
3,247,11325025596515485190,10002759413662457525,PRIVATE_AUTO,WORK,HOME,0 days 07:24:41,0 days 07:37:33,6.867275,nan,530670109103,47.016462,-122.909304,530670112003,47.047701,-122.808723,residential,single_family,residential,single_family,civic_institutional,education,civic_institutional,education,47.016530,-122.908200,47.040000,-122.800000,thursday,3706854826325843051,530670109103,530670112003,nan,53067010910,53067011200,nan,25_34,33.0,F,white,not_hispanic_or_latino,40000_80000,51862.0,employed,high_school,not_attending_school,naics92,spouse,0,nan,driving,owner,same_house,4_person,75000_125000,111288.0,married_couple,3_plus,single_family,core,other,47.016530,-122.908200,47.040000,-122.80000,NaN,NaN,worked_in_person,WORK
4,4243,2725037000145676387,10002759413662457525,PRIVATE_AUTO,SHOP,WORK,0 days 09:52:13,0 days 09:58:59,1.866797,nan,530670112003,47.047701,-122.808723,530670112003,47.047701,-122.808723,civic_institutional,education,civic_institutional,edu

In [3]:
df['destination_building_use_l2'].value_counts()

destination_building_use_l2
single_family               1889
retail                      1839
office                       392
multi_family                 365
education                    183
non_retail_attraction        155
open_space                   105
industrial                   102
civic_institutional           81
healthcare                    55
transportation_utilities      46
agriculture                    8
other                          4
unknown                        3
Name: count, dtype: int64

In [ ]:
if df['destination_building_use_l2'] == 'single_family':
    # most single family homes should be L2 @ 7.2
    pass

elif df['destination_building_use_l2'] == 'multi_family':
    pass